In [34]:
import os
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # 递归字符文本分割器
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma # Chroma向量数据库
from langchain_core.output_parsers import StrOutputParser # 字符输出解析器
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # OpenAI嵌入模型和聊天模型
import tiktoken
import numpy as np
import bs4
from langchain.prompts import ChatPromptTemplate

### API

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ['OPENAI_API_KEY'] = 'sk-2b3073febb6b486e94e62f9e9704f759'
os.environ['EMBEDDING_API_KEY'] = 'sk-zk23347d533c0c49e3d781430f5bb140c0a67d78ee90597e'

In [5]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load() # 加载网页

# 创建文本分割器并分割网页文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("文本分割完成")


文本分割完成


In [ ]:

vectstore = Chroma.from_documents(splits, OpenAIEmbeddings(model="text-embedding-ada-002",
                                                           api_key=os.environ.get('EMBEDDING_API_KEY'),
                                                           base_url="https://api.zhizengzeng.com/v1/",
                                                           encoding_format="float"))
print("向量数据库创建完成")


d:\Programs\miniconda3\envs\RAG\lib\site-packages\langchain_openai\embeddings\base.py:281: UserWarning: WARNING! encoding_format is not default parameter.
                    encoding_format was transferred to model_kwargs.
                    Please confirm that encoding_format is what you intended.
  warnings.warn(


向量数据库创建完成


In [9]:
# 对split进行vectorembedding,并放入vectorstore
retriever = vectstore.as_retriever()

In [10]:
# prompt模板
prompt = hub.pull("rlm/rag-prompt")

# # llm
llm = ChatOpenAI(
    model='deepseek-chat',
    openai_api_key=os.environ.get('OPEN_API_KEY'),
    openai_api_base='https://api.deepseek.com',
    max_tokens=1024
)

# 后处理
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}\n\n" for doc in docs])

# chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or steps. This can be achieved using techniques like Chain of Thought (CoT) or Tree of Thoughts, which guide models to think step-by-step or explore multiple reasoning paths. It helps in planning and executing tasks efficiently by simplifying the overall problem.'

In [12]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [15]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """
    Returns the number of tokens in a text string.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [16]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings(model="text-embedding-ada-002",
                        api_key=os.environ.get('EMBEDDING_API_KEY'),
                        base_url="https://api.zhizengzeng.com/v1/",
                        encoding_format="float")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result), len(document_result)

d:\Programs\miniconda3\envs\RAG\lib\site-packages\langchain_openai\embeddings\base.py:281: UserWarning: WARNING! encoding_format is not default parameter.
                    encoding_format was transferred to model_kwargs.
                    Please confirm that encoding_format is what you intended.
  warnings.warn(


(1536, 1536)

In [18]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915838611735


In [20]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

In [21]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=50
)

splits = text_splitter.split_documents(blog_docs)

In [22]:
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=embd)
retriever = vectorstore.as_retriever()

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000185AE6FD7C0>, search_kwargs={'k': 1})

In [32]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

In [33]:
len(docs)

1

In [35]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\nQuestion: {question}\n'), additional_kwargs={})])

In [36]:
llm = ChatOpenAI(
    model='deepseek-chat',
    openai_api_key=os.environ.get('OPEN_API_KEY'),
    openai_api_base='https://api.deepseek.com',
    max_tokens=1024,
    temperature=0
)

In [37]:
chain = prompt | llm

In [38]:
chain.invoke({
    "context": docs,
    "question": "What is Task Decomposition?"
})

AIMessage(content='Task Decomposition is a process where a complicated task is broken down into smaller and simpler steps. This is often achieved using techniques like Chain of Thought (CoT), which instructs the model to "think step by step" to decompose hard tasks into more manageable components. This approach enhances model performance on complex tasks by utilizing more test-time computation and provides insight into the model\'s thinking process.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 183, 'total_tokens': 261, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 183}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-76aba947-816b-4934-97b0-574a3bc3f11d-0', usage_metadata={'input_tokens': 183, 'output_tokens': 78, 'total

In [39]:
prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [41]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. This is often achieved using techniques like Chain of Thought (CoT), which encourages models to "think step by step" to simplify and tackle each subtask effectively. It helps in planning and interpreting the model\'s reasoning process.'